In [ ]:
import json
import time
import cv2
import numpy as np

# Load JSON file
json_file = "heian_shodan_reference.json"
with open(json_file, 'r') as f:
    data = json.load(f)

# Extract the poses from the nested structure
if isinstance(data, dict) and "steps" in data:
    poses = data["steps"]  # The poses are in the "steps" key
else:
    poses = data  # In case it's a direct array

print(f"Loaded {len(poses)} poses from {json_file}")

# Drawing connections for MediaPipe pose (33 landmarks)
POSE_CONNECTIONS = [
    # Face
    (0, 1), (1, 2), (2, 3), (3, 7),  # Right side face to ear
    (0, 4), (4, 5), (5, 6), (6, 8),  # Left side face to ear
    
    # Shoulders and arms
    (9, 10),  # Mouth
    (11, 12),  # Shoulders
    (11, 13), (13, 15),  # Left arm
    (12, 14), (14, 16),  # Right arm
    
    # Torso
    (11, 23), (12, 24),  # Shoulders to hips
    (23, 24),  # Hip line
    
    # Legs
    (23, 25), (24, 26),  # Hips to knees
    (25, 27), (26, 28),  # Knees to ankles
    
    # Feet
    (27, 29), (28, 30),  # Ankles to heels
    (29, 31), (30, 32),  # Heels to toes
    (27, 31), (28, 32),  # Ankles to toes
]

def draw_pose(landmarks, step_name, image_size=(800, 600)):
    """Draw a pose with proper scaling and centering"""
    img = np.ones((image_size[1], image_size[0], 3), dtype=np.uint8) * 255
    
    # Find the bounds of the pose for better centering
    if not landmarks:
        return img
    
    # Convert landmarks to arrays for easier processing
    points = []
    for lm in landmarks:
        points.append([lm['x'], lm['y']])
    points = np.array(points)
    
    # Calculate bounds
    min_x, min_y = points.min(axis=0)
    max_x, max_y = points.max(axis=0)
    
    # Scale and center the pose
    margin = 50
    scale_x = (image_size[0] - 2 * margin) / (max_x - min_x) if (max_x - min_x) > 0 else 1
    scale_y = (image_size[1] - 2 * margin) / (max_y - min_y) if (max_y - min_y) > 0 else 1
    scale = min(scale_x, scale_y)  # Use same scale for both dimensions
    
    # Center the pose
    center_x = image_size[0] // 2
    center_y = image_size[1] // 2
    pose_center_x = (min_x + max_x) / 2
    pose_center_y = (min_y + max_y) / 2
    
    # Draw connections
    for connection in POSE_CONNECTIONS:
        start_idx, end_idx = connection
        if start_idx < len(landmarks) and end_idx < len(landmarks):
            # Scale and position points
            x1 = int(center_x + (landmarks[start_idx]['x'] - pose_center_x) * scale)
            y1 = int(center_y + (landmarks[start_idx]['y'] - pose_center_y) * scale)
            x2 = int(center_x + (landmarks[end_idx]['x'] - pose_center_x) * scale)
            y2 = int(center_y + (landmarks[end_idx]['y'] - pose_center_y) * scale)
            
            # Only draw if points are within image bounds
            if (0 <= x1 < image_size[0] and 0 <= y1 < image_size[1] and 
                0 <= x2 < image_size[0] and 0 <= y2 < image_size[1]):
                cv2.line(img, (x1, y1), (x2, y2), (0, 100, 200), 3)
    
    # Draw joints
    for i, lm in enumerate(landmarks):
        x = int(center_x + (lm['x'] - pose_center_x) * scale)
        y = int(center_y + (lm['y'] - pose_center_y) * scale)
        
        # Only draw if point is within image bounds
        if 0 <= x < image_size[0] and 0 <= y < image_size[1]:
            # Color code important joints
            if i in [15, 16]:  # Wrists
                cv2.circle(img, (x, y), 8, (0, 0, 255), -1)  # Red for wrists
            elif i in [13, 14]:  # Elbows
                cv2.circle(img, (x, y), 6, (255, 0, 0), -1)  # Blue for elbows
            elif i in [11, 12]:  # Shoulders
                cv2.circle(img, (x, y), 6, (0, 255, 0), -1)  # Green for shoulders
            else:
                cv2.circle(img, (x, y), 4, (100, 100, 100), -1)  # Gray for others
    
    # Add step information
    cv2.putText(img, f"Step: {step_name}", (20, 40), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)
    
    # Add instructions
    cv2.putText(img, "Press SPACE for next, ESC to quit", (20, image_size[1] - 20), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (100, 100, 100), 1)
    
    return img

def main():
    """Main viewer loop"""
    print(" Kata Pose Viewer")
    print("Controls:")
    print("  SPACE/ENTER: Next pose")
    print("  ESC: Quit")
    print("  Any other key: Next pose")
    print("-" * 40)
    
    current_pose = 0
    
    while current_pose < len(poses):
        pose = poses[current_pose]
        step_name = pose["step_name"]
        
        # Get the correct landmarks key
        if "normalized_landmarks" in pose:
            landmarks = pose["normalized_landmarks"]
        elif "landmarks_2d" in pose:
            landmarks = pose["landmarks_2d"]
        else:
            print(f"⚠️ No landmarks found for {step_name}")
            current_pose += 1
            continue
        
        print(f"Showing: {current_pose + 1}/{len(poses)} - {step_name}")
        
        # Draw and display the pose
        frame = draw_pose(landmarks, step_name)
        cv2.imshow("Kata Pose Viewer", frame)
        
        # Wait for key press
        key = cv2.waitKey(0) & 0xFF
        
        if key == 27:  # ESC to quit
            break
        elif key == ord('b'):  # 'b' for back
            current_pose = max(0, current_pose - 1)
        else:  # Any other key for next
            current_pose += 1
    
    cv2.destroyAllWindows()
    print(" Viewer closed")

if __name__ == "__main__":
    main()

Loaded 18 poses from heian_shodan_reference.json
 Kata Pose Viewer
Controls:
  SPACE/ENTER: Next pose
  ESC: Quit
  Any other key: Next pose
----------------------------------------
Showing: 1/18 - Ichi
Showing: 2/18 - Ni
Showing: 3/18 - San
Showing: 4/18 - Shi
Showing: 5/18 - Go
Showing: 6/18 - Roku
Showing: 7/18 - Shichi
Showing: 8/18 - Hachi
Showing: 9/18 - Ku
Showing: 10/18 - Ju
Showing: 11/18 - Ju Ichi
Showing: 12/18 - Ju Ni
Showing: 13/18 - Ju San
Showing: 14/18 - Ju Shi
Showing: 15/18 - Ju Go
Showing: 16/18 - Ju Roku
Showing: 17/18 - Ju Shichi
Showing: 18/18 - Ju Hachi
 Viewer closed


: 